## HW1
### 주어진 ./data/minidata안의 파일 안에서 64x64 사이즈의 고양이 한마리와 128X128 사이즈의 강아지 한마리, 그리고 각각의 파일 명을 내 뱉는 Dataloader를 만드시오
### Data augmentation은 train일 경우 Horizon Flip, 랜덤90도 rotation, 랜덤 Crop을, train이 아닌 경우 하지 않도록 작성하시오

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision import datasets, transforms

from PIL import Image
import os
from glob import glob


class HwDataset(Dataset):
    def __init__(self, train_dir = '/content/drive/MyDrive/CNN_MNC_211224-20211224T050700Z-001/CNN_MNC_211224/data/minidata/train', train = False):
        self.train = train
        self.train_dir = train_dir
        self.train_data = glob(os.path.join(self.train_dir, '*', '*'))
        self.transform_cat = transforms.Compose([transforms.Resize((64, 64)),
                                             transforms.ToTensor(),
                                             ])
        self.transform_dog = transforms.Compose([transforms.Resize((128, 128)),
                                             transforms.ToTensor(),
                                             ])
        self.transform = transforms.Compose([
                         transforms.RandomResizedCrop((224, 224)),
                         transforms.RandomRotation((-90, 90)),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),                                
                              ])
        
    def __len__(self):
        return len(self.train_data)
    
    #  슬라이싱을 구현할 수 있도록 돕는 메소드
    def __getitem__(self, index):
        image = Image.open(self.train_data[index]).convert('RGB')
        # train 이면 transform()
        if self.train :
            image = self.transform()
        else :
            return None

        print(self.train_data[index])

        image_name = self.train_data[index].split('/')[-1] # Linux



        if self.train_data[index].split('/')[-2] == 'cat':
            transforms.Compose([transforms.Resize((64, 64)),
                                             transforms.ToTensor(),
                                             ])


        elif self.train_data[index].split('/')[-2] =='dog': # Linux
            transforms.Compose([transforms.Resize((128, 128)),
                                             transforms.ToTensor(),
                                             ])
            image = self.transform_dog

        return image, image_name

mydataset = HwDataset()
print('Dataset의 개수',len(mydataset))
mydataset

Dataset의 개수 20


In [83]:
train_dataset = HwDataset('/content/drive/MyDrive/CNN_MNC_211224-20211224T050700Z-001/CNN_MNC_211224/data/minidata/train',train=True)
val_dataset = HwDataset('/content/drive/MyDrive/CNN_MNC_211224-20211224T050700Z-001/CNN_MNC_211224/data/minidata/val',train=False)

## HW2
### 2_NeuralNetwork.ipynb의 Let's Do It 코드를 작성해서 결과를 확인하시오
(10000개 이하의 파라미터로 99%의 성능 달성)

In [84]:
# 필요한 train test코드를 다 작성하세요
import torch 
import torch.nn as nn #
import torch.nn.functional as F # 각종 activation 함수
import torchvision # 이미지 관련 처리, Pretrained Model 관련된 Package 입니다. 
import torchvision.datasets as vision_dsets
import torchvision.transforms as T # 이미지 처리 (Vison) 관련된 transformation이 정의 되어 있습니다.
import torch.optim as optim # pytorch 에서 정의한 수 많은 optimization function 들이 들어 있습니다.
from torch.utils import data

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [85]:
def MNIST_DATA(root='/content/drive/MyDrive/CNN_MNC_211224-20211224T050700Z-001/CNN_MNC_211224/data',train =True,transforms=None ,download =True,batch_size = 32,num_worker = 1):
    print ("[+] Get the MNIST DATA")
    """
    torchvision.dataset 에는 우리가 많이 사용하는 데이터들을 쉽게 사용할 수 있도록 되어 있습니다. 
    Mchine Learning 에서 Hello world 라고 불리는 Mnist 데이터를 사용해 보겠습니다. 
    """
    mnist_train = vision_dsets.MNIST(root = root,  #root 는 데이터의 저장 위치 입니다. 
                                    train = True, #Train 은 이 데이터가 train 데이터인지 아닌지에 대한 정보입니다. 
                                    transform = T.ToTensor(), # 얻어낸 데이터를 pytorch가 계산 할 수 있는 Tensor 로 변환해 줍니다. 
                                    download = True)  # 데이터를 다운로드 할지 여부를 물어봅니다. 
    mnist_test = vision_dsets.MNIST(root = root,
                                    train = False,  # Test Data를 가져오기에 Train =False 를 줘야 합니다. 
                                    transform = T.ToTensor(),
                                    download = True)
    """
    Data Loader 는 데이터와 batch size의 정보를 바탕으로 매 iteration 마다 주어진 데이터를 원하는 batch size 만큼 반환해주는 iterator입니다. 
    * Practical Guide : Batch size 는 어느정도가 좋나요? -- 클 수록 좋다는 소리가 있습니다. 하지만 gpu memeory 사이즈 한계에 의해 기본적으로 batch size 가 
      커질 수록 학습에 사용되는 gpu memory 사이즈가 큽니다. (Activation map을 저장해야 하기 때문입니다.) 기본적으로 2의 배수로 저장하는 것이 좋습니다.(Bit size 관련) 
    """
    trainDataLoader = data.DataLoader(dataset = mnist_train,  # DataSet은 어떤 Data를 제공해 줄지에 대한 정보입니다. 여기서는 Training DATA를 제공합니다. 
                                      batch_size = batch_size, # batch size 정보를 꼭 줘야 합니다. 한 Batch 당 몇 개의 Data 를 제공할지에 대한 정보입니다. 
                                      shuffle =True, # Training의 경우 Shuffling 을 해주는 것이 성능에 지대한 영향을 끼칩니다. 꼭 True 를 줘야 합니다. 
                                      num_workers = 1) # num worker의 경우 데이터를 로드하는데 worker를 얼마나 추가하겠는가에 대한 정보입니다. 

    testDataLoader = data.DataLoader(dataset = mnist_test, # Test Data Loader 이므로 Test Data를 인자로 전달해줍니다.
                                    batch_size = batch_size, # 마찬가지로 Batch size 를 넣어줍니다. 
                                    shuffle = False, # shuffling 이 굳이 필요하지 않으므로 false를 줍니다. 
                                    num_workers = 1) #
    print ("[+] Finished loading data & Preprocessing")
    return mnist_train,mnist_test,trainDataLoader,testDataLoader

In [86]:
trainDset,testDset,trainDataLoader,testDataLoader= MNIST_DATA(batch_size = 32)

[+] Get the MNIST DATA
[+] Finished loading data & Preprocessing


In [87]:
class Trainer():
    def __init__(self, trainloader, testloader, net, optimizer, criterion):
        """
        trainloader: train data의 loader 입니다
        testloader: test data의 loader 입니다
        net: 학습시킬 모델입니다
        optimizer: 모델의 파라미터를 업데이트할 최적화 함수입니다
        criterion: 모델의 loss function 입니다.
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.net = net
        self.optimizer = optimizer
        self.criterion = criterion
        
    def train(self, epoch = 1):
        """
        epoch: 전체 학습 데이터의 사용횟수입니다.
        """
        self.net.train()
        for e in range(epoch):
            running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
            for i, data in enumerate(self.trainloader, 0): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
                # get the inputs
                inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
                inputs = inputs.cuda()
                labels = labels.cuda()
                # zero the parameter gradients
                self.optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다. 
                                              #  Q) 이걸 안 한다면?

                # forward + backward + optimize
                outputs = self.net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
                loss = self.criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
                loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
                self.optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

                # print statistics
                running_loss += loss.item()
                if (i+1) % 500 == 0:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' % (e + 1, i + 1, running_loss / 500))
                    running_loss = 0.0

        print('Finished Training')
        
    def test(self):
        self.net.eval() # Eval Mode 왜 해야 할까요?  
                        # --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
        test_loss = 0
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs.cuda()
            labels = labels.cuda() 
            output = self.net(inputs) 
            pred = output.max(1, keepdim=True)[1] # get the index of the max 
            correct += pred.eq(labels.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

            test_loss /= len(self.testloader.dataset)
        print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.
                format(correct, len(self.testloader.dataset),
                100.* correct / len(self.testloader.dataset)))

In [88]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.InstanceNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [89]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [90]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [91]:
trainer.train(epoch = 4)

[1,   500] loss: 0.577
[1,  1000] loss: 0.226
[1,  1500] loss: 0.170
[2,   500] loss: 0.131
[2,  1000] loss: 0.127
[2,  1500] loss: 0.110
[3,   500] loss: 0.097
[3,  1000] loss: 0.091
[3,  1500] loss: 0.090
[4,   500] loss: 0.080
[4,  1000] loss: 0.082
[4,  1500] loss: 0.076
Finished Training


In [92]:
trainer.test()


Test set:  Accuracy: 9778/10000 (98%)



In [93]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mnist_net)


3186

## (Optional) HW3
### 3_VGG.ipynb를 참조해 VGG16을 구현하세요